In [ ]:
import os
import pickle
import sys

import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import cm
from scipy.integrate import simps, trapz
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

In [ ]:
from dask.distributed import Client

client = Client(n_workers=8)
client

In [ ]:
!ls /n/groups/paulsson/jqs1/molecule-counting/

In [ ]:
with open(
    "/n/groups/paulsson/jqs1/molecule-counting/200302photobleaching.pickle", "rb"
) as f:
    d = pickle.load(f)
    print(f"File is {sys.getsizeof(f)/1e6} MB")
list(d.keys())

In [ ]:
from dask import delayed

In [ ]:
def decay_rates(data, t_end):
    return data.apply(
        lambda y: (y.values[0] - y.values[t_end - 1])
        / np.sum(y[:t_end].values - y.values[t_end - 1]),
        axis=1,
    ).values


def process_df(full_data, bad_fovs=[], trunc=0, dy_max=10, dy_min=0):
    traces_list = []
    rp_list = []
    for fp_dir in full_data.keys():
        print(f"FP directory is {fp_dir}")
        for i, fov in enumerate(list(full_data[fp_dir].keys())):
            if i not in bad_fovs:
                print(f"FOV {i}")
                for channel in full_data[fp_dir][fov]["traces"].keys():
                    print(f"Channel {channel}")
                    regionprops = full_data[fp_dir][fov]["regionprops"]
                    timeseries = pd.DataFrame(
                        full_data[fp_dir][fov]["traces"][channel]["mean"][1:, 1:]
                    )

                    traces_list.append(timeseries)

                    regionprops["FOV"] = i
                    regionprops["experiment"] = os.path.basename(
                        os.path.dirname(fp_dir)
                    )
                    regionprops["channel"] = channel
                    rp_list.append(regionprops)

                    print(timeseries.shape)

    traces = pd.concat(traces_list)
    df_meta = pd.concat(rp_list, sort=False)
    df_meta.reset_index(inplace=True)
    print(df_meta.columns)
    print(traces.shape)

    df_data = pd.DataFrame(traces)
    #     data = pd.concat([data,rp_df],axis=1,sort=False)
    print("computing k")
    t_end = traces.shape[1] - trunc
    cell_id = df_meta.index.tolist()
    df_meta["cell_id"] = cell_id
    df_meta["k"] = decay_rates(df_data, t_end)
    dt = 10
    df_meta["dy"] = df_data.apply(
        lambda y: np.abs((y.values[: t_end - dt] / y.values[dt:t_end])).max(), axis=1
    )
    df_n = df_data.apply(lambda x: x / x[0], axis=1)
    print("hv stuff")
    time = np.arange(df_data.shape[1])
    data = hv.Table(
        (time, cell_id, df_data, df_n), ["time", "cell_id"], ["fluor", "normed"]
    )  # TODO Fix that

    kdims = ["centroid-0", "centroid-1"]
    vdims = [
        "label",
        "area",
        "eccentricity",
        "min_intensity",
        "mean_intensity",
        "max_intensity",
        "major_axis_length",
        "minor_axis_length",
        "orientation",
        "perimeter",
        "solidity",
        "weighted_centroid-0",
        "weighted_centroid-1",
        "FOV",
        "cell_id",
        "k",
        "dy",
    ]
    meta_data = hv.Table(df_meta, kdims, vdims)
    return data, meta_data

In [ ]:
mkate_dir = "200227/mKate2hyb_genomic/*.nd2"
data, meta_data = process_df(d, [], 6, 1.3, 0.90)

In [ ]:
def flatten(d):
    